In [ ]:
from tools import *

In [ ]:
import datetime

print(fetch_stock_price("GOOG",
                        datetime.date(2017,1,1),
                        datetime.date(2017,1,31)))

## visualization

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn
from tools import fetch_cosine_values, fetch_stock_price, format_dataset

np.set_printoptions(precision=2)

# # Print 2 oscillations (20 points, with a frequency of 0.1, i.e. a period of 10)
cos_values = fetch_cosine_values(20, frequency=0.1)
seaborn.pointplot(cos_values)
plt.xlabel("Days since start of the experiment")
plt.ylabel("Value of the cosine function")
plt.title("Cgit push"
          "osine time series over time")
plt.show()

In [ ]:
# Format the feature and label set
features_size = 5
minibatch_cos_X, minibatch_cos_y = format_dataset(cos_values, features_size)
print("minibatch_cos_X.shape=", minibatch_cos_X.shape)
print("minibatch_cos_y.shape=", minibatch_cos_y.shape)

In [ ]:
minibatch_cos_X